<a href="https://colab.research.google.com/github/seulmi0827/fininsight/blob/main/Jace/BERTopic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q bertopic
!pip install -q bertopic[visualization]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.6 MB/s eta 0:00:00


In [3]:
!apt-get update
!apt-get install g++ openjdk-8-jdk -y
!pip install konlpy
!pip install mecab-python
!apt-get install curl -y
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package fonts-dejavu-core.
(Reading database ... 126213 files and directories currently installed.)
Preparing to unpack .../00-fonts-dejavu-core_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-core (2.37-2build1) ...
Selecting previously unselected package fonts-dejavu-extra.
Preparing to unpack .../01-fonts-dejavu-extra_2.37-2build1_all.deb ...
Unpacking fonts-dejavu-extra (2.37-2build1) ...
Selecting previously unselected package libfontenc1:amd64.
Preparing to unpack .../02-libfontenc1_1%3a1.1.4-1build3_amd64.deb ...
Unpacking libfontenc1:amd64 (1:1.1.4-1build3) ...
Selecting previously unselected package libxkbfile1:amd64.
Preparing to unpack .../03-libxkbfile1_1%3a1.1.0-1build3_amd64.deb ...
Unpacking libxkbfile1:amd64 (1:1.1.0-1build3) ...
Selecting previously un

# 전처리

In [12]:
from konlpy.tag import Mecab
import pandas as pd

mecab = Mecab()

def preprocess(text):
    # 명사 추출
    nouns = mecab.nouns(text)

    # 형태소 분석
    pos_tagged = mecab.pos(text)

    # 동사 추출 (VV: 동사)
    verbs = [word for word, pos in pos_tagged if pos.startswith('VV')]

    # 형용사 추출 (VA: 형용사)
    adjectives = [word for word, pos in pos_tagged if pos.startswith('VA')]

    # 부사 추출 (MAG: 일반부사)
    adverbs = [word for word, pos in pos_tagged if pos == 'MAG']

    # 모든 단어 결합 (1글자 이상인 것만)
    all_words = nouns + verbs + adjectives + adverbs
    filtered_words = [word for word in all_words if len(word) > 1]

    # 공백으로 조인하여 문자열로 반환
    return filtered_words


df = pd.read_csv("/content/drive/MyDrive/fin/삼성전자_10000.csv")
df['content'] = df['content'].fillna('').astype(str)

original_content = df['content'].tolist()

preprocessed_content = [preprocess(preprocessed_content) for preprocessed_content in original_content]

In [13]:
len(preprocessed_content)

10000

# Mecab과 SBERT를 이용한 Bertopic

In [5]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from hdbscan import HDBSCAN

import torch
import numpy as np

import random

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
  torch.cuda.manual_seed_all(SEED)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

cuda


In [15]:
umap_model = UMAP(
   n_neighbors=15,
   n_components=5,
   min_dist=0.0,
   metric='cosine',
   random_state=SEED
)

hdbscan_model = HDBSCAN(
   min_cluster_size=5,
   metric='euclidean',
   prediction_data=True,
   gen_min_span_tree=True,
   cluster_selection_method='eom'
)


embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

vectorizer = CountVectorizer(stop_words=None) # TF-IDF

topic_model = BERTopic(
   language="korean",
   nr_topics=50,
   top_n_words=10,
   calculate_probabilities=True,
   umap_model=umap_model,
   hdbscan_model=hdbscan_model,
   embedding_model=embedding_model,
   vectorizer_model=vectorizer, # TF-IDF to c-TF-IDF
   verbose=True, # 로그 추적
)


content_for_topic = [' '.join(doc) for doc in preprocessed_content]
topics, probs = topic_model.fit_transform(content_for_topic)


# topics는 각 문서가 할당된 주요 토픽의 ID를 담은 배열입니다.
# probs는 각 문서가 모든 토픽에 대해 가지는 확률값을 담은 2D 배열입니다. 크기는 (문서 수 × 토픽 수)입니다.

# calculate_probabilities 옵션의 차이는 다음과 같습니다:
  # True로 설정 시:

  # 각 문서가 각 토픽에 속할 확률을 계산합니다
  # fit_transform()이 토픽 ID와 확률값 두 가지를 반환합니다: topics, probs = fit_transform()
  # 문서가 여러 토픽에 걸쳐 있는 정도를 알 수 있습니다
  # 계산 시간이 더 오래 걸립니다

  # False로 설정 시(기본값):
  # 각 문서를 가장 가능성 높은 하나의 토픽에만 할당합니다
  # fit_transform()이 토픽 ID만 반환합니다: topics = fit_transform()
  # 계산이 더 빠릅니다
  # 다중 토픽 정보는 얻을 수 없습니다

2025-04-07 09:52:15,577 - BERTopic - Embedding - Transforming documents to embeddings.


Batches:   0%|          | 0/313 [00:00<?, ?it/s]

2025-04-07 09:52:31,576 - BERTopic - Embedding - Completed ✓
2025-04-07 09:52:31,577 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-04-07 09:53:12,605 - BERTopic - Dimensionality - Completed ✓
2025-04-07 09:53:12,606 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-04-07 09:55:19,466 - BERTopic - Cluster - Completed ✓
2025-04-07 09:55:19,467 - BERTopic - Representation - Extracting topics using c-TF-IDF for topic reduction.
2025-04-07 09:55:21,318 - BERTopic - Representation - Completed ✓
2025-04-07 09:55:21,320 - BERTopic - Topic reduction - Reducing number of topics
2025-04-07 09:55:21,344 - BERTopic - Representation - Fine-tuning topics using representation models.
2025-04-07 09:55:22,781 - BERTopic - Representation - Completed ✓
2025-04-07 09:55:22,784 - BERTopic - Topic reduction - Reduced number of topics from 505 to 50


In [16]:
print(topic_model.get_topic_info())
print('='*50)
topic_info = topic_model.get_topic_info()
total_documents = topic_info['Count'].sum()
print(f"총 문서 수: {total_documents}")

# Topic: 토픽 ID 번호 (-1은 아웃라이어 토픽)
# Count: 토탈 문서 수 중, 각 토픽에 할당된 문서 수
# Name: 토픽의 자동 생성된 이름 (주요 키워드로 구성)
# Representation: 토픽을 대표하는 주요 단어들 (top_n_words=k에 해당)
# Representative_Docs: 각 토픽을 가장 잘 대표하는 문서 예시

# 아웃라이너 : BERTopic에서 토픽 ID가 -1로 표시되는 특별한 분류
# 주요 토픽들에 잘 맞지 않는 문서들의 집합 (일관된 주제가 없거나 여러 주제가 혼합된 문서들이 포함, 아주 독특한 내용을 가진 문서들이 포함)
# BERTopic은 HDBSCAN 클러스터링을 사용하는데, 이 알고리즘은 노이즈가 많거나 데이터 밀도가 낮은 영역의 포인트를 아웃라이어로 분류

    Topic  Count                 Name  \
0      -1   1310      -1_반도체_미국_중국_기업   
1       0   1213      0_갤럭시_시리즈_구독_애플   
2       1    986    1_반도체_메모리_하이닉스_분기   
3       2    986      2_사업_이사_삼성전자_주주   
4       3    696        3_기술_기업_시크_통신   
5       4    593       4_포크_비스_에어컨_기능   
6       5    507       5_관세_미국_트럼프_부과   
7       6    453       6_올트먼_회장_오픈_회동   
8       7    373       7_주식_삼성_금융_지난해   
9       8    272    8_갤럭시_스튜디오_체험_방문객   
10      9    246        9_기판_유리_소재_인증   
11     10    237       10_회장_검찰_무죄_선고   
12     11    220      11_디자인_평가_부문_선정   
13     12    192    12_경제_사절단_회장_대한상의   
14     13    176     13_지수_코스피_하락_외국인   
15     14    169       14_채용_청년_삼성_교육   
16     15    147    15_부회장_빈소_고인_장례식장   
17     16    116  16_로봇_휴머노이드_로보_레인보우   
18     17    111    17_아트_게임_오디세이_모니터   
19     18     87      18_보락_정보_청소기_개인   
20     19     83      19_주주_상법_개정안_기업   
21     20     71     20_홈플러스_대금_납품_회생   
22     21     68       21_임금_합의_노사_삼노   
23     22     68

In [17]:
for topic_id in topic_model.get_topics():
    print(f"Topic {topic_id}: {topic_model.get_topic(topic_id)}")

Topic -1: [('반도체', np.float64(0.012491818826156277)), ('미국', np.float64(0.011604476854205733)), ('중국', np.float64(0.011397128020805933)), ('기업', np.float64(0.011076933746828506)), ('시장', np.float64(0.010411489828065045)), ('삼성전자', np.float64(0.010047842245770393)), ('회장', np.float64(0.00910949014677719)), ('기술', np.float64(0.009018106269746971)), ('한국', np.float64(0.00872730229681685)), ('투자', np.float64(0.008651145805147826))]
Topic 0: [('갤럭시', np.float64(0.040586198771156476)), ('시리즈', np.float64(0.023979630503771547)), ('구독', np.float64(0.01877852757790055)), ('애플', np.float64(0.018425088205974376)), ('스마트폰', np.float64(0.018231361426124536)), ('기능', np.float64(0.017096059483907394)), ('삼성전자', np.float64(0.015741903906731077)), ('모델', np.float64(0.015292588329936959)), ('사용', np.float64(0.014953638855081787)), ('출시', np.float64(0.014903656360139395))]
Topic 1: [('반도체', np.float64(0.027300499586543753)), ('메모리', np.float64(0.024562606708645893)), ('하이닉스', np.float64(0.024179461537246

In [18]:
barchart = topic_model.visualize_barchart()
barchart.show()
print('\n')

heatmap = topic_model.visualize_heatmap()
heatmap.show()
print('\n')

hierarchy = topic_model.visualize_hierarchy()
hierarchy.show()
print('\n')

term_rank = topic_model.visualize_term_rank()
term_rank.show()
print('\n')

topics = topic_model.visualize_topics()
topics.show()